# 🖼️  SUP OS LEVEL 1 ComfyUI GGUF Runner (GOD VERSION)

### 📌 Instruções Críticas:
1. **Reinicie o Ambiente:** Vá em **Ambiente de execução > Desconectar e excluir ambiente de execução** antes de começar.
2. **Célula 1:** Instala tudo e corrige as versões das bibliotecas.
3. **Célula 3:** Inicia o servidor (agora com limpeza automática de porta).
4. The models downloaded via the notebook are in "comfyui/models" organize it before open the comfyui otherwise you can't do shit :(
5. use LocalTunnel(more faster) or Cloudflare

Adaptado para máxima compatibilidade.

In [1]:
#@title 1️⃣ Instalar Dependências e Fixar Versões { display-mode: "form" }
from IPython.display import clear_output, display, HTML
import os
import subprocess

def print_with_style(message, style='info'):
    colors = {'info': '#31708f', 'success': '#3c763d', 'warning': '#8a6d3b', 'error': '#a94442'}
    backgrounds = {'info': '#d9edf7', 'success': '#dff0d8', 'warning': '#fcf8e3', 'error': '#f2dede'}
    color = colors.get(style, 'black')
    background = backgrounds.get(style, 'white')
    display(HTML(f'<div style="border: 1px solid {color}; color: {color}; background-color: {background}; margin: 10px; padding: 10px; border-radius: 5px;">{message}</div>'))

print_with_style("🧹 Limpando ambiente...", "warning")
!apt-get install -y psmisc
# Remove conflitos
!pip uninstall -y torch torchvision torchaudio transformers huggingface_hub

print_with_style("🔄 Instalando PyTorch e Bibliotecas...", "info")
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

%cd /content
if not os.path.exists('ComfyUI'):
    !git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
!git pull

!pip install -q -r requirements.txt
# Fix crítico para HuggingFace
!pip install -q -U "huggingface_hub>=0.34.0" transformers requests gguf

!mkdir -p ./models/diffusion_models ./models/text_encoders ./models/vae ./models/unet ./custom_nodes ./models/other

%cd ./custom_nodes
repos = {
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "ComfyUI-GGUF": "https://github.com/city96/ComfyUI-GGUF.git"
}
for name, url in repos.items():
    if not os.path.exists(name):
        !git clone {url}
    else:
        %cd {name}
        !git pull
        %cd ..

clear_output()
print_with_style("✅ Instalação concluída.", "success")

In [ ]:
#@title 2️⃣ Baixar Modelos e Arquivos do Hugging Face { display-mode: "form" }
from huggingface_hub import hf_hub_download, list_repo_files, get_hf_file_metadata
from collections import defaultdict
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import re

repo_url = "lym00/Wan2.1_T2V_1.3B_SelfForcing_VACE" #@param {type:"string"}
modelo_predefinido = "Selecione um modelo pré-definido ou insira um link acima" #@param ["Selecione um modelo pré-definido ou insira um link acima", "calcuis/hidream-gguf", "calcuis/kontext-gguf", "calcuis/cosmos-predict2-gguf", "calcuis/wan-1.3b-gguf"]

def print_with_style(message, style='info'):
    colors = {'info': '#31708f', 'success': '#3c763d', 'warning': '#8a6d3b', 'error': '#a94442'}
    backgrounds = {'info': '#d9edf7', 'success': '#dff0d8', 'warning': '#fcf8e3', 'error': '#f2dede'}
    color = colors.get(style, 'black')
    background = backgrounds.get(style, 'white')
    display(HTML(f'<div style="border: 1px solid {color}; color: {color}; background-color: {background}; margin: 10px; padding: 10px; border-radius: 5px;">{message}</div>'))

def get_file_size_str(repo_id, filename):
    try:
        metadata = get_hf_file_metadata(repo_id=repo_id, filename=filename)
        size_bytes = metadata.size
        if size_bytes < 1024 * 1024:
            return f"{size_bytes / 1024:.1f} KB"
        elif size_bytes < 1024 * 1024 * 1024:
            return f"{size_bytes / (1024 * 1024):.1f} MB"
        else:
            return f"{size_bytes / (1024 * 1024 * 1024):.1f} GB"
    except Exception:
        return "Tamanho desconhecido"

if modelo_predefinido != "Selecione um modelo pré-definido ou insira um link acima":
    repo_url = modelo_predefinido

if not repo_url:
    print_with_style("⚠️ Por favor, insira um link de repositório do Hugging Face ou selecione um modelo pré-definido.", "warning")
else:
    if "huggingface.co/" in repo_url:
        repo_id = repo_url.split("huggingface.co/")[1]
    else:
        repo_id = repo_url
    repo_id = re.sub(r'/tree/.*$|/blob/.*$|/files/.*$', '', repo_id).rstrip('/')

    print_with_style(f"🔄 Conectando ao repositório: <strong>{repo_id}</strong>...", 'info')

    try:
        all_files = list_repo_files(repo_id)
        if not all_files:
            print_with_style(f"⚠️ Nenhum arquivo encontrado no repositório {repo_id}", "warning")
        else:
            print_with_style(f"Buscando detalhes de {len(all_files)} arquivos... Isso pode levar um momento.", 'info')
            file_details = {f: get_file_size_str(repo_id, f) for f in all_files}

            clear_output(wait=True)
            print_with_style(f"✅ Encontrados {len(all_files)} arquivos no repositório.", "success")

            categorized_files = defaultdict(list)
            for f, size in file_details.items():
                ext = os.path.splitext(f)[1].lower()
                if ext == '.gguf': categorized_files['GGUF'].append((f, size))
                elif ext == '.json': categorized_files['JSON (Workflows)'].append((f, size))
                elif ext == '.safetensors': categorized_files['Safetensors'].append((f, size))
                else: categorized_files['Outros'].append((f, size))

            all_checkboxes_map = {}
            ui_sections = []

            for category, files in categorized_files.items():
                if not files: continue
                header = widgets.HTML(f"<div style='margin-top: 10px;'><b>{category} disponíveis:</b></div>")
                checkboxes = []
                for file_path, size_str in sorted(files):
                    cb = widgets.Checkbox(value=True, description=f"{os.path.basename(file_path)} ({size_str})", style={'description_width': 'initial'}, layout=widgets.Layout(width='auto'))
                    checkboxes.append(cb)
                    all_checkboxes_map[file_path] = cb
                ui_sections.extend([header, widgets.VBox(checkboxes)])

            select_all_button = widgets.Button(description="Selecionar Todos", button_style='info', icon='check-square-o')
            deselect_all_button = widgets.Button(description="Desselecionar Todos", button_style='warning', icon='square-o')
            download_button = widgets.Button(description='Baixar Selecionados', button_style='success', icon='download')

            def set_all_checkboxes(value, b):
                for cb in all_checkboxes_map.values(): cb.value = value
            select_all_button.on_click(lambda b: set_all_checkboxes(True, b))
            deselect_all_button.on_click(lambda b: set_all_checkboxes(False, b))

            def download_selected_files(b):
                buttons = [b, select_all_button, deselect_all_button]
                for btn in buttons: btn.disabled = True

                selected_files = [fp for fp, cb in all_checkboxes_map.items() if cb.value]
                if not selected_files:
                    print_with_style("⚠️ Nenhum arquivo selecionado.", "warning")
                else:
                    for file_path in selected_files:
                        filename = os.path.basename(file_path)
                        ext = os.path.splitext(filename)[1].lower()
                        save_dir = "/content/ComfyUI/models/other"

                        if ext == '.json':
                            save_dir = "/content/ComfyUI"
                        elif ext in ['.gguf', '.safetensors']:
                            if any(x in filename.lower() for x in ['clip', 't5xxl', 'llama']): save_dir = "/content/ComfyUI/models/text_encoders"
                            elif any(x in filename.lower() for x in ['vae', 'pig']): save_dir = "/content/ComfyUI/models/vae"
                            elif any(x in filename.lower() for x in ['unet']): save_dir = "/content/ComfyUI/models/unet"
                            else: save_dir = "/content/ComfyUI/models/diffusion_models"

                        print_with_style(f"Baixando <strong>{filename}</strong> para <em>{save_dir}</em>...", 'info')
                        try:
                            hf_hub_download(repo_id=repo_id, filename=file_path, local_dir=save_dir, local_dir_use_symlinks=False, resume_download=True)
                            print_with_style(f"✅ <strong>{filename}</strong> baixado com sucesso!", "success")
                        except Exception as e:
                            print_with_style(f"❌ Erro ao baixar {filename}: {e}", "error")
                    print_with_style("✅ Download de arquivos concluído!", "success")

                for btn in buttons: btn.disabled = False

            download_button.on_click(download_selected_files)

            display(widgets.VBox(ui_sections))
            display(widgets.HBox([select_all_button, deselect_all_button, download_button]))

    except Exception as e:
        print_with_style(f"❌ Erro ao acessar o repositório: {e}", "error")

In [ ]:
#@title 🎨 Baixar Modelos do Civitai (com Seleção Múltipla) { display-mode: "form" }
#@markdown Cole a URL da página de um modelo do Civitai para listar e baixar seus arquivos.
civitai_url = "https://civitai.com/models/166609/realism-by-stable-yogi?modelVersionId=1223643"  #@param {type:"string"}

import requests
import re
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import shutil # Usado para verificar a existência de comandos

# --- Funções de Ajuda ---
def print_with_style(message, style='info'):
    """Exibe uma mensagem formatada com um estilo visual."""
    colors = {'info': '#31708f', 'success': '#3c763d', 'warning': '#8a6d3b', 'error': '#a94442'}
    backgrounds = {'info': '#d9edf7', 'success': '#dff0d8', 'warning': '#fcf8e3', 'error': '#f2dede'}
    color = colors.get(style, 'black')
    background = backgrounds.get(style, 'white')
    display(HTML(f'<div style="border: 1px solid {color}; color: {color}; background-color: {background}; margin: 10px; padding: 10px; border-radius: 5px;">{message}</div>'))

def format_size(size_kb):
    """Formata o tamanho do arquivo de KB para uma string legível (KB, MB, GB)."""
    if size_kb is None: return "Tamanho desconhecido"
    if size_kb < 1024: return f"{size_kb:.1f} KB"
    if size_kb < 1024 * 1024: return f"{size_kb / 1024:.1f} MB"
    return f"{size_kb / (1024 * 1024):.1f} GB"

def get_civitai_model_info(url):
    """Busca os dados de um modelo na API do Civitai usando sua URL."""
    match = re.search(r'civitai.com/models/(\d+)', url)
    if not match:
        print_with_style("URL do Civitai inválida. Use o formato 'https://civitai.com/models/12345/...'", "error")
        return None
    model_id = match.group(1)
    api_url = f"https://civitai.com/api/v1/models/{model_id}"
    try:
        response = requests.get(api_url, headers={'User-Agent': 'colab-downloader/1.0'})
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print_with_style(f"Erro ao acessar a API do Civitai: {e}", "error")
        return None

# --- Lógica Principal ---

# **CORREÇÃO APLICADA AQUI**
# Verifica e instala o aria2c se necessário, antes de qualquer outra coisa.
if not shutil.which('aria2c'):
    print_with_style("O downloader 'aria2c' não foi encontrado. Instalando...", 'warning')
    # O -qq suprime a maior parte da saída para manter o output limpo
    !apt-get -y install -qq aria2
    print_with_style("'aria2c' instalado com sucesso!", 'success')

if not civitai_url:
    print_with_style("Por favor, insira uma URL do Civitai.", "warning")
else:
    clear_output(wait=True)
    print_with_style("Buscando informações do modelo no Civitai...", "info")
    model_data = get_civitai_model_info(civitai_url)

    if model_data:
        clear_output(wait=True)
        model_name = model_data.get('name', 'N/A')
        model_type = model_data.get('type', 'Unknown')
        print_with_style(f"Modelo encontrado: <b>{model_name}</b> | Tipo: <b>{model_type}</b>", "success")

        # Extrai todos os arquivos de todas as versões
        all_files_data = []
        for version in model_data.get('modelVersions', []):
            version_name = version.get('name', 'N/A')
            for file_info in version.get('files', []):
                all_files_data.append({
                    'name': file_info['name'],
                    'size_str': format_size(file_info.get('sizeKB')),
                    'url': file_info['downloadUrl'],
                    'type': model_type,
                    'version_name': version_name
                })

        # --- Constrói a Interface Interativa ---
        all_checkboxes = [
            widgets.Checkbox(value=True, description=f"<b>{f['name']}</b> ({f['size_str']}) - <i>Versão: {f['version_name']}</i>",
                             style={'description_width': 'initial'},
                             layout=widgets.Layout(width='95%'))
            for f in all_files_data
        ]

        select_all_button = widgets.Button(description="Selecionar Todos", button_style='info', icon='check-square-o')
        deselect_all_button = widgets.Button(description="Desselecionar Todos", button_style='warning', icon='square-o')
        download_button = widgets.Button(description='Baixar Selecionados', button_style='success', icon='download')

        button_box = widgets.HBox([select_all_button, deselect_all_button, download_button])
        ui = widgets.VBox([widgets.VBox(all_checkboxes), button_box])

        # --- Lógica dos Botões ---
        def set_all_checkboxes(value, b):
            for cb in all_checkboxes:
                cb.value = value

        select_all_button.on_click(lambda b: set_all_checkboxes(True, b))
        deselect_all_button.on_click(lambda b: set_all_checkboxes(False, b))

        def download_civitai_files(b):
            for btn in [select_all_button, deselect_all_button, download_button]: btn.disabled = True

            selected_files = [all_files_data[i] for i, cb in enumerate(all_checkboxes) if cb.value]
            if not selected_files:
                print_with_style("Nenhum arquivo selecionado.", "warning")
            else:
                TYPE_TO_DIR = {
                    'Checkpoint': 'models/checkpoints',
                    'LORA': 'models/loras',
                    'LoCon': 'models/loras',
                    'VAE': 'models/vae',
                    'TextualInversion': 'models/embeddings'
                }

                for f in selected_files:
                    dest_dir_name = TYPE_TO_DIR.get(f['type'], 'models/other')
                    # Assume que a pasta do ComfyUI está em /content/ComfyUI
                    full_dest_path = os.path.join("/content/ComfyUI", dest_dir_name)
                    os.makedirs(full_dest_path, exist_ok=True)

                    print_with_style(f"👇 Baixando <b>{f['name']}</b> para <i>{full_dest_path}</i>...", 'info')
                    # Usando aria2c para download rápido e com continuação de download
                    !aria2c -c -x 16 -s 16 -k 1M --console-log-level=warn -d '{full_dest_path}' -o "{f['name']}" "{f['url']}"
                    print_with_style(f"✅ Download de <b>{f['name']}</b> concluído!", 'success')

            for btn in [select_all_button, deselect_all_button, download_button]: btn.disabled = False

        download_button.on_click(download_civitai_files)

        display(ui)

In [3]:
#@title 3️⃣ Iniciar ComfyUI (Túnel Nativo Colab) { display-mode: "form" }
#@markdown ⚠️ **AVISO:** Este modo usa o túnel oficial do Google. Ele é mais estável, mas requer que o ComfyUI aceite CORS.
import subprocess
import threading
import time
import re
import os
from IPython.display import display, HTML
from google.colab.output import eval_js

metodo_acesso = "LocalTunnel" #@param ["Túnel do Colab", "Cloudflared", "LocalTunnel"]
PORT = 8188
HOME_DIR = "/content/ComfyUI"

def print_with_style(message, style='info'):
    colors = {'info': '#31708f', 'success': '#3c763d', 'warning': '#8a6d3b', 'error': '#a94442'}
    backgrounds = {'info': '#d9edf7', 'success': '#dff0d8', 'warning': '#fcf8e3', 'error': '#f2dede'}
    color = colors.get(style, 'black')
    background = backgrounds.get(style, 'white')
    display(HTML(f'<div style="border: 1px solid {color}; color: {color}; background-color: {background}; margin: 10px; padding: 10px; border-radius: 5px;">{message}</div>'))

print_with_style("🛡️ Limpando processos antigos na porta 8188...", "warning")
subprocess.run(["fuser", "-k", "8188/tcp"])
subprocess.run(["pkill", "-f", "cloudflared"])
subprocess.run(["pkill", "-f", "lt"])

os.chdir(HOME_DIR)

# Arguments cruciais para o Túnel do Colab funcionar
cmd_args = [
    "python", "main.py",
    "--listen", "0.0.0.0",
    "--port", str(PORT),
    "--enable-cors-header" # IMPORTANTE: Permite que o proxy do Colab acesse o servidor
]

print_with_style(f"🚀 Iniciando ComfyUI com suporte a CORS... Aguarde o carregamento.")
comfyui_proc = subprocess.Popen(cmd_args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True, bufsize=1)

server_ready_event = threading.Event()

def stream_output(proc, ready_event):
    for line in iter(proc.stdout.readline, ''):
        line = line.strip()
        if line:
            print(f"[ComfyUI Log] {line}")
            if "To see the GUI go to:" in line:
                if not ready_event.is_set():
                    ready_event.set()
    proc.stdout.close()

reader_thread = threading.Thread(target=stream_output, args=(comfyui_proc, server_ready_event))
reader_thread.daemon = True
reader_thread.start()

print_with_style("Aguardando servidor iniciar (timeout 120s)...", "info")
ready = server_ready_event.wait(timeout=120)

if not ready:
    print_with_style("❌ O ComfyUI não iniciou a tempo. Verifique a Célula 1.", "error")
else:
    print_with_style("✅ Servidor Iniciado! Gerando link...", "success")
    url = None

    if metodo_acesso == "Túnel do Colab":
        try:
            url = eval_js(f'google.colab.kernel.proxyPort({PORT})')
        except Exception as e:
            print_with_style(f"Erro Túnel Colab: {e}", "error")

    elif metodo_acesso == "Cloudflared":
        !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared
        !chmod +x /content/cloudflared
        print("Iniciando Cloudflared...")
        !nohup /content/cloudflared tunnel --url http://127.0.0.1:8188 > cloudflared.log 2>&1 &
        time.sleep(5)
        try:
            for i in range(15):
                if os.path.exists('cloudflared.log'):
                    with open('cloudflared.log', 'r') as f:
                        logs = f.read()
                        if ".trycloudflare.com" in logs:
                            url = re.search(r'https?://[a-zA-Z0-9-]+\.trycloudflare\.com', logs).group(0)
                            break
                time.sleep(2)
        except Exception as e:
            print(f"Erro Cloudflare: {e}")

    elif metodo_acesso == "LocalTunnel":
        print("Iniciando LocalTunnel...")
        import urllib.request
        public_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
        print_with_style(f"🔑 <b>SENHA DO LOCALTUNNEL:</b> <span style='font-size: 20px; color: red; font-weight: bold;'>{public_ip}</span> (Copie este IP)", "warning")
        !nohup npx localtunnel --port 8188 > localtunnel.log 2>&1 &
        time.sleep(5)
        try:
             for i in range(15):
                if os.path.exists('localtunnel.log'):
                    with open('localtunnel.log', 'r') as f:
                        logs = f.read()
                        if "your url is:" in logs:
                            url = logs.split("your url is: ")[1].strip()
                            break
                time.sleep(2)
        except Exception as e:
             print(f"Erro LocalTunnel: {e}")

    if url:
        display(HTML(f"<div style='padding: 20px; border-radius: 5px; border: 2px solid #4CAF50; background-color: #e8f5e9; margin: 10px 0px;'> <h2 style='color: #2e7d32; text-align: center;'>Acesso ao ComfyUI Pronto!</h2> <p style='font-size: 16px; text-align: center;'>Clique no link abaixo:</p> <p style='text-align: center;'><a href='{url}' target='_blank' style='font-size: 20px; padding: 12px 24px; background-color: #4CAF50; color: white; text-decoration: none; border-radius: 5px; display: inline-block; margin-top: 10px; font-weight: bold;'>{url}</a></p> </div>"))
        if metodo_acesso == "Túnel do Colab":
             print("💡 Dica: Se der 404, aguarde 10 segundos e recarregue a página.")
    else:
        print_with_style("❌ Falha ao obter URL pública.", "error")

    print_with_style("A célula continuará rodando para manter o servidor ativo. Os logs serão exibidos acima. Não interrompa a execução.")

[ComfyUI Log] [START] Security scan
[ComfyUI Log] [DONE] Security scan
[ComfyUI Log] ## ComfyUI-Manager: installing dependencies done.
[ComfyUI Log] ** ComfyUI startup time: 2026-01-07 15:51:58.201
[ComfyUI Log] ** Platform: Linux
[ComfyUI Log] ** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
[ComfyUI Log] ** Python executable: /usr/bin/python3
[ComfyUI Log] ** ComfyUI Path: /content/ComfyUI
[ComfyUI Log] ** ComfyUI Base Folder Path: /content/ComfyUI
[ComfyUI Log] ** User directory: /content/ComfyUI/user
[ComfyUI Log] ** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
[ComfyUI Log] ** Log path: /content/ComfyUI/user/comfyui.log
[ComfyUI Log] Prestartup times for custom nodes:
[ComfyUI Log] 6.5 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager
[ComfyUI Log] Checkpoint files will always be loaded safely.
[ComfyUI Log] Total VRAM 15095 MB, total RAM 12976 MB
[ComfyUI Log] pytorch version: 2.9.1+cu128
[ComfyUI Log] Set vram state to: NORMAL

Iniciando LocalTunnel...


[ComfyUI Log] FETCH ComfyRegistry Data: 5/118
[ComfyUI Log] FETCH ComfyRegistry Data: 10/118


In [ ]:
#@title 📊 Monitorar Status da GPU (Opcional)
# Executa o comando nvidia-smi a cada 10 segundos para monitorar o uso da GPU.
# Interrompa a execução desta célula para parar o monitoramento.
!nvidia-smi -l 10

In [ ]:
#@title 🛌 Manter o Colab Ativo (Opcional) { display-mode: "form" }
#@markdown Execute esta célula para impedir que o Google Colab desconecte por inatividade.
#@markdown Ela entrará em um loop infinito. Você pode interrompê-la a qualquer momento sem afetar o ComfyUI.
import time
print("✅ Loop de manutenção iniciado. O Colab não será desconectado por inatividade.")
print("Pressione o botão de 'Stop' (⏹️) ao lado da célula para parar este loop.")
while True:
    time.sleep(300) # Dorme por 5 minutos